In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-05 20:00:47.715636: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-05 20:00:47.749643: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 20:00:47.749681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 20:00:47.750964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 20:00:47.756958: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-05 20:00:47.757474: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)

      print("train dataset:", np.shape(self.train_dataset), self.train_dataset)
      print("val dataset:", np.shape(self.val_dataset), self.val_dataset)

 
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [20]:
import pickle
from itertools import product

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)



# Treinando com 25 dados


In [23]:
tester = Tester(run_times=25, dataset_run_times=10)

In [24]:
tester.Loop(sort_by='mse_sup',
            boundarie=2.0,
            hidden_sizes = [[30, 16], [24, 7], [15, 2]],
            regularizers=[0.02, 0.2, 0.1],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[30, 16], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  23.311360006999166
Stopped at epoch:  960
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.15929069200137747
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.8234635419994447
Stopped at epoch:  32
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.20464548699965235
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.38638036100019235
Stopped at epoch:  16
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.38645282900142774
Stopped at epoch:  14
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  1.3012803719993826
Stopped at epoch:  53
1/1 [==============================] - 0s 14ms/

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_3_5,0.928820,0.709371,0.983159,0.975739,0.984068,0.298726,1.943435,0.088241,0.112050,0.100146
model_1_3_4,0.927784,0.707796,0.983713,0.976557,0.984599,0.303074,1.953967,0.085341,0.108270,0.096806
model_1_3_3,0.926715,0.706195,0.984254,0.977348,0.985115,0.307562,1.964673,0.082506,0.104619,0.093563
model_1_3_2,0.925611,0.704568,0.984782,0.978110,0.985616,0.312193,1.975555,0.079738,0.101098,0.090418
model_1_3_1,0.924473,0.702914,0.985297,0.978844,0.986100,0.316970,1.986614,0.077037,0.097708,0.087373
model_1_3_0,0.923299,0.701234,0.985800,0.979549,0.986568,0.321896,1.997852,0.074406,0.094452,0.084429


DataFrame salvo em ../results/metrics_1_3
+++++++++++ [1_4] | 1 ++++++++++++++++++
Elapsed time:  0.1672267579997424
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_4] | 2 ++++++++++++++++++
Elapsed time:  0.1608646629993018
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 3 ++++++++++++++++++
Elapsed time:  0.18496819199936
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_4] | 4 ++++++++++++++++++
Elapsed time:  0.16264997799953562
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 5 ++++++++++++++++++
Elapsed time:  0.17594099400048435
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_4] | 6 ++++++++++++++++++
Elapsed time:  0.16367660400101158
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 7 ++++++++++++++++++
Elapsed time:  0.16447529200

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_4_24,0.934380,0.716302,0.995436,-0.085078,0.867628,0.275395,1.897090,0.025168,1.148862,0.587015
model_1_4_23,0.934345,0.716169,0.995477,-0.075660,0.868778,0.275538,1.897978,0.024942,1.138891,0.581916
model_1_4_22,0.934298,0.716026,0.995517,-0.066355,0.869914,0.275738,1.898935,0.024720,1.129039,0.576879
model_1_4_21,0.934237,0.715872,0.995557,-0.057165,0.871035,0.275994,1.899963,0.024503,1.119308,0.571905
model_1_4_20,0.934162,0.715708,0.995595,-0.048092,0.872142,0.276308,1.901062,0.024291,1.109702,0.566996
model_1_4_19,0.934073,0.715533,0.995633,-0.039142,0.873234,0.276683,1.902235,0.024083,1.100225,0.562154
model_1_4_18,0.933969,0.715346,0.995670,-0.030315,0.874311,0.277117,1.903483,0.023880,1.090880,0.557380
model_1_4_17,0.933851,0.715148,0.995706,-0.021615,0.875372,0.277614,1.904807,0.023681,1.081669,0.552675
model_1_4_16,0.933717,0.714938,0.995741,-0.013047,0.876417,0.278174,1.906209,0.023487,1.072598,0.548042
model_1_4_15,0.933568,0.714717,0.995775,-0.004612,0.877445,0.278799,1.907690,0.023297,1.063666,0.543482


...

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_7_4_24,0.967382,0.787480,0.510436,0.977725,0.925628,0.136890,1.421120,0.619863,0.081067,0.350465
model_7_4_23,0.967353,0.787335,0.512364,0.977704,0.925879,0.137013,1.422093,0.617422,0.081144,0.349283
model_7_4_22,0.967322,0.787187,0.514312,0.977682,0.926132,0.137144,1.423084,0.614955,0.081226,0.348091
model_7_4_21,0.967289,0.787036,0.516281,0.977657,0.926387,0.137282,1.424094,0.612462,0.081315,0.346889
model_7_4_20,0.967254,0.786882,0.518270,0.977632,0.926644,0.137427,1.425123,0.609944,0.081409,0.345676
model_7_4_19,0.967218,0.786725,0.520280,0.977604,0.926903,0.137580,1.426173,0.607399,0.081509,0.344454
model_7_4_18,0.967179,0.786565,0.522312,0.977575,0.927165,0.137741,1.427242,0.604827,0.081615,0.343221
model_7_4_17,0.967139,0.786402,0.524364,0.977544,0.927429,0.137909,1.428332,0.602228,0.081728,0.341978
model_7_4_16,0.967097,0.786236,0.526438,0.977511,0.927695,0.138086,1.429443,0.599602,0.081848,0.340725
model_7_4_15,0.967053,0.786066,0.528533,0.977476,0.927963,0.138271,1.430575,0.596949,0.081974,0.339461


DataFrame salvo em ../results/metrics_7_4
+++++++++++ [7_5] | 1 ++++++++++++++++++
Elapsed time:  0.7577059159993951
Stopped at epoch:  33
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_5] | 2 ++++++++++++++++++
Elapsed time:  0.15355366299991147
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [7_5] | 3 ++++++++++++++++++
Elapsed time:  0.16399089799961075
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [7_5] | 4 ++++++++++++++++++
Elapsed time:  0.1533212860013009
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_5] | 5 ++++++++++++++++++
Elapsed time:  0.20056274899980053
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [7_5] | 6 ++++++++++++++++++
Elapsed time:  0.1820541819997743
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [7_5] | 7 ++++++++++++++++++
Elapsed time:  0.22469008

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_7_5_0,0.970952,0.785823,0.987679,0.914195,0.967628,0.121908,1.432203,0.095252,0.494105,0.294678
model_7_5_1,0.970990,0.785734,0.987353,0.914207,0.967494,0.121750,1.432797,0.097769,0.494034,0.295901
model_7_5_2,0.971023,0.785644,0.987024,0.914216,0.967357,0.121610,1.433399,0.100311,0.493983,0.297147
model_7_5_3,0.971052,0.785553,0.986692,0.914221,0.967218,0.121488,1.434008,0.102879,0.493951,0.298415
model_7_5_4,0.971077,0.785461,0.986357,0.914224,0.967076,0.121383,1.434623,0.105468,0.493938,0.299703
model_7_5_5,0.971098,0.785368,0.986020,0.914223,0.966933,0.121295,1.435244,0.108079,0.493942,0.301010
model_7_5_6,0.971115,0.785275,0.985680,0.914219,0.966787,0.121222,1.435870,0.110709,0.493964,0.302336
model_7_5_7,0.971129,0.785180,0.985337,0.914213,0.966640,0.121165,1.436502,0.113357,0.494001,0.303679
model_7_5_8,0.971139,0.785085,0.984992,0.914203,0.966490,0.121122,1.437138,0.116021,0.494055,0.305038
model_7_5_9,0.971146,0.784989,0.984646,0.914192,0.966339,0.121094,1.437779,0.118701,0.494123,0.306412


DataFrame salvo em ../results/metrics_7_5
+++++++++++ [7_6] | 1 ++++++++++++++++++
Elapsed time:  0.16061464400081604
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_6] | 2 ++++++++++++++++++
Elapsed time:  0.15637480100122048
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step


: 